In [21]:
import importlib
from IPython.display import display

import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from pandas.tseries.offsets import Day, MonthEnd
from datetime import datetime
from datetime import timedelta
from datetime import date

from account import Index, Account, Merge

In [23]:
idx = Index('2021-04', periods=12, freq='M')
idx.mtrt = 12

In [37]:
ntnl = Account(idx, "LoanA_ntnl", ("LoanA", "ntnl"))
ntnl.amt = 1000
ntnl.subscdd(idx.index[0], ntnl.amt)
ntnl.subamt(idx.index[0], ntnl.df.loc[idx.index[0], 'sub_rsdl_cum'])
ntnl.addamt(idx.index[11], ntnl.amt)

fee = Account(idx, "LoanA_fee", ("LoanA", "fee"))
fee.rate = 0.01
fee.amt = ntnl.amt * fee.rate
fee.addscdd(idx.index[0], fee.amt)
fee.addamt(idx.index[0], fee.df.loc[idx.index[0], 'add_rsdl_cum'])

IR = Account(idx, "LoanA_IR", ("LoanA", "IR"))
IR.rate = 0.05
IR.amt = ntnl.amt * IR.rate / 12
IR.addscdd(idx.index, np.ones(len(idx)) * IR.amt)
IR.addamt(idx.index, np.ones(len(idx)) * IR.amt)

dct = {"ntnl":ntnl, "fee":fee, "IR":IR}
LoanA = Merge(dct)

In [52]:
LoanA.dct['fee'].addamt(idx.index[6], 20)

In [53]:
LoanA.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
2021-04-30,14.166667,14.166667,1000.0,1000.0,0.000000,14.166667,14.166667,1000.0,1000.0,-985.833333,0.0,0.0
2021-05-31,4.166667,18.333333,0.0,1000.0,-985.833333,4.166667,18.333333,0.0,1000.0,-981.666667,0.0,0.0
2021-06-30,4.166667,22.500000,0.0,1000.0,-981.666667,4.166667,22.500000,0.0,1000.0,-977.500000,0.0,0.0
2021-07-31,4.166667,26.666667,0.0,1000.0,-977.500000,4.166667,26.666667,0.0,1000.0,-973.333333,0.0,0.0
2021-08-31,4.166667,30.833333,0.0,1000.0,-973.333333,4.166667,30.833333,0.0,1000.0,-969.166667,0.0,0.0
2021-09-30,4.166667,35.000000,0.0,1000.0,-969.166667,4.166667,35.000000,0.0,1000.0,-965.000000,0.0,0.0
2021-10-31,4.166667,39.166667,0.0,1000.0,-965.000000,24.166667,59.166667,0.0,1000.0,-940.833333,-20.0,0.0
2021-11-30,4.166667,43.333333,0.0,1000.0,-940.833333,4.166667,63.333333,0.0,1000.0,-936.666667,-20.0,0.0
2021-12-31,4.166667,47.500000,0.0,1000.0,-936.666667,4.166667,67.500000,0.0,1000.0,-932.500000,-20.0,0.0
2022-01-31,4.166667,51.666667,0.0,1000.0,-932.500000,4.166667,71.666667,0.0,1000.0,-928.333333,-20.0,0.0


In [93]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from pandas.tseries.offsets import Day, MonthEnd
import datetime as dt
from datetime import datetime
from datetime import timedelta
from datetime import date
from functools import wraps

import genfunc
from index import Index
from account import Account, Merge


class Loan(object):
    def __init__(self,
                 index = None, # Index class
                 amt_ntnl = None, # float
                 rate_fee = None, # float
                 rate_IR = None, # float
                 title = None, # string : "LoanA"
                 tag = None, # string tuple : ("tagA", "tagB")
                 note = "" # string
                 ):
        # index 입력
        if isinstance(index, Index):
            self.cindex = index
            self.index = index.index
            
        # 주요 변수 입력
        self.amt_ntnl = amt_ntnl
        self.rate_fee = rate_fee
        self.rate_IR = rate_IR
            
        # title 입력
        self.title = title
        
        # tag 입력 : tag는 튜플로 받음. string으로 입력된 경우 튜플로 변환 필요
        if isinstance(tag, tuple):
            self.tag = tag
        elif isinstance(tag, str):
            self.tag = tuple(tag)
        elif tag is None:
            self.tag = None
        else:
            raise ValueError("tag is not a tuple")
            
        # note 입력
        self.note = note
        
        # Account Setting
        self.ntnl = Account(self.cindex, self.title, self.tag)
        self.fee = Account(self.cindex, self.title, self.tag)
        self.IR = Account(self.cindex, self.title, self.tag)
        
        # Initialize
        self._intlz()
        
    def _intlz(self):
        # 초기화 함수 실행
        dct = {}
        
        self.ntnl.amt = self.amt_ntnl
        self.ntnl.subscdd(self.cindex.index[0], self.ntnl.amt)
        self.ntnl.addscdd(self.cindex.index[-1], self.ntnl.amt)
        dct['ntnl'] = self.ntnl
        
        self.fee.rate = self.rate_fee
        self.fee.amt = self.ntnl.amt * self.fee.rate
        self.fee.addscdd(self.cindex.index[0], self.fee.amt)
        dct['fee'] = self.fee
        
        self.IR.rate = self.rate_IR
        self.IR.amt = self.ntnl.amt * self.IR.rate
        self.IR.addscdd(self.cindex.index, np.ones(len(self.cindex)) * self.IR.amt)
        dct['IR'] = self.IR
        
        self.loan = Merge(dct)
        

In [94]:
k = Loan(idx, 1000, 0.01, 0.05, "LoanA", ("Loan", "A"))

In [95]:
k.IR.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
2021-04-30,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0
2021-05-31,50.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
2021-06-30,50.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0
2021-07-31,50.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0
2021-08-31,50.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0
2021-09-30,50.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,0.0
2021-10-31,50.0,350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,0.0
2021-11-30,50.0,400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0
2021-12-31,50.0,450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,450.0,0.0
2022-01-31,50.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0


In [96]:
k.loan.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
2021-04-30,60.0,60.0,1000.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,1000.0
2021-05-31,50.0,110.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,1000.0
2021-06-30,50.0,160.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0,1000.0
2021-07-31,50.0,210.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,210.0,1000.0
2021-08-31,50.0,260.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,260.0,1000.0
2021-09-30,50.0,310.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,310.0,1000.0
2021-10-31,50.0,360.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,1000.0
2021-11-30,50.0,410.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,410.0,1000.0
2021-12-31,50.0,460.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,460.0,1000.0
2022-01-31,50.0,510.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,510.0,1000.0


In [97]:
k.fee.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
2021-04-30,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-05-31,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-06-30,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-07-31,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-08-31,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-09-30,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-10-31,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-11-30,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2021-12-31,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2022-01-31,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
